In [1]:
# Чем чаще cтудент совершает действия на курсе, тем выше средняя промежуточная оценка студентов.
# Таблица логов за 2022-2023 год для ДО с ролью 5 (Студент ДО): DO(2022-2023)
# журнал оценок по ипр, кр, тестам для ДО: Grade.csv
# выгрузка айдишников преподавателей, в том числе преподавателей-магистрантов: lmsdb_mdl_user

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import pytz
import matplotlib as mpl
import calendar
import statistics

In [3]:
# Открываем "журнал оценок" и преобразвываем дату
d_parser = lambda x: pd.to_datetime(x, unit='s')
Grade = pd.read_csv('C://Users//37529//Desktop//data since//2th sem//Grade.csv', quotechar="'", delim_whitespace=True,
                    parse_dates=['timemodified'], date_parser=d_parser, index_col=0)
Grade['timemodified'] = Grade['timemodified'] + pd.Timedelta(hours=3)


In [4]:
# выбираем определенный период времени для исследования (с 01.09.2022-1.03.2023)
Grade_2022_2023=Grade.loc[(Grade['timemodified']>='2022-09-01')&(Grade['timemodified']<'2023-01-31')]
# Сортировка по времени
Grade_2022_2023 = Grade_2022_2023.sort_values(by='timemodified')
# Делаем выборку по оценкам от (0 до 10). Оценки, которым принадлежат от (10-100) не будут участвовать в анализе.
all_marks=list(Grade.finalgrade.unique())
selected_all_marks=[]
for i in all_marks:
    if i<=10.0:
        selected_all_marks.append(i)
Grade_2022_2023 = Grade_2022_2023[Grade_2022_2023["finalgrade"].isin(selected_all_marks)]
# удаляем ненужные колонки
Grade_2022_2023=Grade_2022_2023.drop(columns = ['name', 'shortname','itemname'], axis = 1) 
Grade_2022_2023.reset_index(drop= True , inplace= True )
# меняем название колонки для дальнейшего объединения двух таблиц
Grade_2022_2023.rename(columns={'finalgrade':'action','timemodified':'timecreated'}, inplace=True)
# создаем список всех курсов на которых были поставлены оценки 
grade_courseid=list(Grade_2022_2023.courseid.unique())
Grade_2022_2023

,courseid,action,timecreated
0,2259,5.00000,2022-09-01 11:17:04
1,71,7.50000,2022-09-02 09:24:56
2,153,10.00000,2022-09-04 11:00:38
3,153,10.00000,2022-09-04 11:06:36
4,153,8.57143,2022-09-04 11:12:53
...,...,...,...
1559,4099,4.00000,2023-01-30 22:19:50
1560,4098,1.20000,2023-01-30 23:50:32
1561,4098,2.00000,2023-01-30 23:50:32
1562,4098,1.00000,2023-01-30 23:56:13


In [5]:
# загрузка  "журнала событий" с записями действий только студентов дистанционной формы обучения
d_parser = lambda x: pd.to_datetime(x, unit='s')
logstore = pd.read_csv('C://Users//37529//Desktop//data since//2th sem//DO(2022-2023).csv', quotechar="'", 
                       delim_whitespace=True, parse_dates=['timecreated'], date_parser=d_parser, index_col=0)
# Преобразование времени
logstore['timecreated'] = logstore['timecreated'] + pd.Timedelta(hours=3)
logstore

,eventname,component,action,target,objecttable,objectid,crud,edulevel,contextid,contextlevel,contextinstanceid,userid,courseid,relateduserid,anonymous,other,timecreated,origin,ip,realuserid
id,,,,,,,,,,,,,,,,,,,,
97412724,\core\event\course_viewed,core,viewed,course,-,-,r,2,10040,50,217,15900,217,-,0,N;,2022-09-13 23:21:32,web,87.249.132.198,-
97412727,\tool_usertours\event\tour_started,tool_usertours,started,tour,tool_usertours_tours,28,r,2,10040,50,217,15900,217,-,0,"a:1:{s:7:""pageurl"";s:43:""https://lms.bsuir.by/...",2022-09-13 23:21:33,web,87.249.132.198,-
97412731,\tool_usertours\event\tour_ended,tool_usertours,ended,tour,tool_usertours_tours,28,c,2,10040,50,217,15900,217,-,0,"a:3:{s:7:""pageurl"";s:43:""https://lms.bsuir.by/...",2022-09-13 23:21:35,web,87.249.132.198,-
96584264,\core\event\course_viewed,core,viewed,course,-,-,r,2,266102,50,4088,24681,4088,-,0,N;,2022-09-02 20:22:48,web,134.17.140.150,-
96584265,\tool_usertours\event\tour_started,tool_usertours,started,tour,tool_usertours_tours,28,r,2,266102,50,4088,24681,4088,-,0,"a:1:{s:7:""pageurl"";s:44:""https://lms.bsuir.by/...",2022-09-02 20:22:50,web,134.17.140.150,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114571311,\mod_book\event\course_module_viewed,mod_book,viewed,course_module,book,180,r,2,163770,70,46534,24761,2250,-,0,N;,2023-03-01 13:14:36,web,37.215.34.108,-
114571312,\mod_book\event\chapter_viewed,mod_book,viewed,chapter,book_chapters,871,r,2,163770,70,46534,24761,2250,-,0,N;,2023-03-01 13:14:36,web,37.215.34.108,-
114571313,\mod_book\event\course_module_viewed,mod_book,viewed,course_module,book,180,r,2,163770,70,46534,24761,2250,-,0,N;,2023-03-01 13:14:36,web,37.215.34.108,-


In [7]:
# сортируем по нужным нам курсам для анализа (в которых есть оценки)
logstore_students=logstore.query("courseid in @grade_courseid")
# оставляем нужные нам столбцы
final_logstore_students=logstore_students[["courseid","action","timecreated"]]
final_logstore_students=final_logstore_students.loc[(final_logstore_students['timecreated']>='2022-09-01')&(final_logstore_students['timecreated']<'2023-01-31')]
final_logstore_students.reset_index(drop= True , inplace= True )
# для дальнейшего анализа меняем значение в колонке 'action' на 'actions of students', которое поможет нам в дальнейшем анализе
final_logstore_students['action'] = final_logstore_students['action'].replace(list(final_logstore_students.action.unique()), 'actions of students')
final_logstore_students

,courseid,action,timecreated
0,217,actions of students,2022-09-13 23:21:32
1,217,actions of students,2022-09-13 23:21:33
2,217,actions of students,2022-09-13 23:21:35
3,4088,actions of students,2022-09-02 20:22:48
4,4088,actions of students,2022-09-02 20:22:50
...,...,...,...
1127136,2250,actions of students,2023-01-30 22:35:29
1127137,2250,actions of students,2023-01-30 22:35:29
1127138,2250,actions of students,2023-01-30 22:35:29
1127139,2250,actions of students,2023-01-30 23:35:54


In [8]:
# Соединяем таблицу с оценками (Grade_2022_2023) с таблицой действий студентов (final_logstore_students)
final_df = pd.concat([Grade_2022_2023.assign(), final_logstore_students.assign()])
# # Сортировка по времени
sorted_df = final_df.sort_values(by='courseid')
# удаление колонки со временем
sorted_df_1=sorted_df[["courseid","action"]]
sorted_df_1

,courseid,action
434289,5,actions of students
437136,5,actions of students
437135,5,actions of students
437134,5,actions of students
437133,5,actions of students
...,...,...
624700,5594,actions of students
624701,5594,actions of students
624702,5594,actions of students
624692,5594,actions of students


# можно сделать по месяцам тип как мы в тестовом задании делали
##взять данные по конец января

In [10]:
sorted_df_2=sorted_df[["courseid","action"]]
sorted_df_2=sorted_df_2.groupby (sorted_df_2.columns.tolist (), as_index= False ). size ()
sorted_df_2.loc[25:50]

,courseid,action,size
25,10,8.0,9
26,10,8.15385,1
27,10,8.25,1
28,10,8.34616,1
29,10,8.42308,1
30,10,8.44231,1
31,10,8.46154,1
32,10,8.53846,1
33,10,8.61539,2
34,10,8.73077,1


In [42]:
# выдделение определенного курса. Нахождение средней ценки за курс и количества действий, совершенных преподавателем 
all_courseid=list(sorted_df.courseid.unique())
df_for_correl=pd.DataFrame({'mark': [], 'actions of teachers': []})
for k in all_courseid:
    dist_с = [k]
    df_one_course = sorted_df_1[sorted_df_1["courseid"].isin(dist_с)]
    all_marks=[]
    actions=0
    for i, row in df_one_course.iterrows():
        if  row['action']=='actions of teachers':
            actions+=1
        else:
            all_marks.append(row['action'])
    everenge_mark=sum(all_marks)/len(all_marks)
    df_for_correl.loc[len(df_for_correl.index)]=[everenge_mark, actions]

#нахождение корреляции 
correlation= df_for_correl['mark'].corr(df_for_correl['actions of teachers'])
correlation

,mark,actions of teachers
0,7.531250,2911.0
1,7.526366,105.0
2,7.213305,4038.0
3,4.719523,701.0
4,6.830070,77.0
...,...,...
127,9.204546,528.0
128,4.000000,297.0
129,5.687372,73.0
130,7.000000,130.0


In [43]:
correlation= df_for_correl['mark'].corr(df_for_correl['actions of teachers'])
correlation

-0.06248718732913578

In [44]:
ne_akk.loc[:50]

,mark,actions of teachers
0,7.531250,2911.0
1,7.526366,105.0
2,7.213305,4038.0
3,4.719523,701.0
4,6.830070,77.0
5,6.311405,3848.0
6,7.000000,1943.0
7,7.699874,5959.0
8,5.776667,740.0
9,8.618026,1400.0


In [ ]:
#Пример кода не аккумулированного подсчета количества просмотров теории для пользователя с id=144
dist_u = [144]
file_ne_akk = sorted_df[sorted_df["userid"].isin(dist_u)]
ne_akk=pd.DataFrame({'osenka': [], 'teoria': []})
teor=0
for i, row in file_ne_akk.iterrows():
    if  row['finalgrade']==11.0:
        teor+=1
    else:
        ne_akk.loc[len(ne_akk.index)]=[row['finalgrade'], teor]
        teor=0
ne_akk

In [46]:
dist_u = [118]
file_ne_akk1 = sorted_df_1[sorted_df_1["courseid"].isin(dist_u)]
file_ne_akk1

,courseid,action
